# File insegtion and schema validation

Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of
read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns
 
file size

In [1]:
import os
import time

In [2]:
os.path.getsize('E:/week6/usedcar.csv')

1282666305

Read Data with Dask

In [3]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('E:/week6/usedcar.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.10300803184509277 sec


Read Data with Panda

In [4]:
import pandas as pd
start = time.time()
df = pd.read_csv('E:/week6/usedcar.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

C:\Users\Nabeel\AppData\Local\Temp\ipykernel_10488\4153505647.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('E:/week6/usedcar.csv')


Read csv with pandas:  166.93011617660522 sec


So here panda taking too much time for 1.2GB file so Dask is better to use

In [5]:
from dask import dataframe as dd
df = dd.read_csv('E:/week6/usedcar.csv',delimiter=',')

In [6]:
df.info()


<class 'dask.dataframe.core.DataFrame'>
Columns: 21 entries, id to zip
dtypes: object(16), float64(2), int64(3)

In [7]:
len(df.index)

C:\Users\Nabeel\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\io\csv.py:193: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\Nabeel\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\io\csv.py:193: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\Nabeel\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\io\csv.py:193: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\Nabeel\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\io\csv.py:193: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\Nabeel\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\io\csv.py:193: DtypeWarning: C

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+---------+----------+
| Column | Found   | Expected |
+--------+---------+----------+
| miles  | float64 | int64    |
| zip    | object  | int64    |
+--------+---------+----------+

The following columns also raised exceptions on conversion:

- zip
  ValueError('cannot convert float NaN to integer')

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'miles': 'float64',
       'zip': 'object'}

to the call to `read_csv`/`read_table`.

In [ ]:
len(df.columns)

Removing white spce from column

In [ ]:
df.columns = df.columns.str.replace(' ', '')

In [ ]:
data=df.columns
data

In [ ]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [ ]:
%%writefile utility.py
import pyyaml

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [ ]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: usedcar
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - id
      - vin
      - price
      - mile
      - stock_no
      - year
      - make
      - model
      - trim
      - body_type
      - vehicle_type
      - drivetrain
      - transmission
      - fuel_type
      - engine_size
      - engine_block
      - seller_name
      - street


In [ ]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('E:/week6/usedcar.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('Rate.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')


In [ ]:
#number of files in gz format folder
import os
entries = os.listdir('E:/week6/usedcar.csv')
for entry in entries:
    print(entry)

In [ ]:
#size of the gz format folder
os.path.getsize('E:/week6usedcar.csv.gz')